In [1]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report
import xgboost as xgb

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
                        SELECT DISTINCT
                            match_date as date,
                            team as home_team,
                            opp_team as away_team,
                            summary_goals as home_goals,
                            opp_summary_goals as away_goals,
                            summary_xg as home_xg,
                            opp_summary_xg as away_xg,
                            keeper_psxg as home_psxg,
                            opp_keeper_psxg as away_psxg,
                            summary_shots as home_shots,
                            opp_summary_shots as away_shots,
                            possession_touches_att_3rd as home_ft_touches,
                            opp_possession_touches_att_3rd as away_ft_touches
                        FROM 
                            fbref_match_summary_v2 ms
                        WHERE
                            ms.division = 'Premier League' AND ms.season = '2024-2025'
                            AND is_home = 1
                       """, conn)

df

,date,home_team,away_team,home_goals,away_goals,home_xg,away_xg,home_psxg,away_psxg,home_shots,away_shots,home_ft_touches,away_ft_touches
0,2025-05-25,Tottenham,Brighton,1.0,4.0,2.0,2.2,1.5,4.1,3.0,22.0,46.0,240.0
1,2025-05-25,Bournemouth,Leicester City,2.0,0.0,1.6,0.3,2.1,0.0,20.0,3.0,226.0,72.0
2,2025-05-25,Newcastle Utd,Everton,0.0,1.0,1.2,1.2,1.7,1.1,17.0,14.0,219.0,73.0
3,2025-05-25,Fulham,Manchester City,0.0,2.0,1.3,3.1,1.3,2.3,13.0,19.0,103.0,182.0
4,2025-05-25,Nott'ham Forest,Chelsea,0.0,1.0,1.2,1.1,0.1,1.1,10.0,6.0,154.0,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2024-08-17,Nott'ham Forest,Bournemouth,1.0,1.0,1.4,1.2,1.7,1.2,14.0,13.0,206.0,160.0
376,2024-08-17,Everton,Brighton,0.0,3.0,0.5,1.4,0.0,2.4,9.0,10.0,105.0,125.0
377,2024-08-17,Newcastle Utd,Southampton,1.0,0.0,0.3,1.8,0.6,0.4,3.0,19.0,72.0,270.0
378,2024-08-17,Ipswich Town,Liverpool,0.0,2.0,0.5,2.6,0.2,2.3,7.0,18.0,98.0,200.0
